# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [18]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [19]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [20]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers, verify=False)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [25]:
ed = Website("https://edwarddonner.com")
# print(ed.get_contents())
ed.links

/opt/anaconda3/envs/llms/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'edwarddonner.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'ht

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [26]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [27]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [28]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [29]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2024/12/21/

In [30]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [31]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

/opt/anaconda3/envs/llms/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/meta-llama/Llama-4-Scout-17B-16E-Instruct',
 '/meta-llama/Llama-4-Maverick-17B-128E-Instruct',
 '/deepseek-ai/DeepSeek-V3-0324',
 '/reducto/RolmOCR',
 '/Qwen/Qwen2.5-Omni-7B',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/jamesliu1217/EasyControl_Ghibli',
 '/spaces/VAST-AI/TripoSG',
 '/spaces/Stable-X/Hi3DGen',
 '/spaces/victor/deepsite-gallery',
 '/spaces',
 '/datasets/nvidia/OpenCodeReasoning',
 '/datasets/open-thoughts/OpenThoughts2-1M',
 '/datasets/nvidia/Llama-Nemotron-Post-Training-Dataset',
 '/datasets/LLM360/MegaMath',
 '/datasets/agentica-org/DeepCoder-Preview-Dataset',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft

In [32]:
get_links("https://huggingface.co")

/opt/anaconda3/envs/llms/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'links': [{'type': 'home page', 'url': 'https://huggingface.co/'},
  {'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'community forum', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [33]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [34]:
print(get_all_details("https://huggingface.co"))

/opt/anaconda3/envs/llms/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/anaconda3/envs/llms/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'discuss page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


/opt/anaconda3/envs/llms/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/anaconda3/envs/llms/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/anaconda3/envs/llms/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/anaconda3/envs/llms/lib

Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
meta-llama/Llama-4-Scout-17B-16E-Instruct
Updated
about 9 hours ago
•
133k
•
669
meta-llama/Llama-4-Maverick-17B-128E-Instruct
Updated
about 10 hours ago
•
17.8k
•
259
deepseek-ai/DeepSeek-V3-0324
Updated
14 days ago
•
175k
•
2.47k
reducto/RolmOCR
Updated
7 days ago
•
2.52k
•
250
Qwen/Qwen2.5-Omni-7B
Updated
9 days ago
•
111k
•
1.29k
Browse 1M+ models
Spaces
Running
3.59k
3.59k
DeepSite
🐳
Generate any application with DeepSeek
Running
on
Zero
877
877
EasyControl Ghibli
🦀
New Ghibli EasyControl model is now released!!
Running
on
Zero
465
465
TripoSG
🔮
Generate 3D models from images
Running
on
Zero
425
425
H

In [45]:
# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


In [46]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [47]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

/opt/anaconda3/envs/llms/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/anaconda3/envs/llms/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'discuss page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}


/opt/anaconda3/envs/llms/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/anaconda3/envs/llms/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apply.workable.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/anaconda3/envs/llms/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/anaconda3/envs/llms

'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nmeta-llama/Llama-4-Scout-17B-16E-Instruct\nUpdated\nabout 9 hours ago\n•\n133k\n•\n669\nmeta-llama/Llama-4-Maverick-17B-128E-Instruct\nUpdated\nabout 10 hours ago\n•\n17.8k\n•\n259\ndeepseek-ai/DeepSeek-V3-0324\nUpdated\n14 days ago\n•\n175k\n•\n2.47k\nreducto/RolmOCR\nUpdated\n7 days ago\n•\n2.52k\n•\n251\nQwen/Qwen2.5-Omni-7B\nUpdated\n9 days ago\n•\n111k\n•\n1.29k\nBrowse 1M+ models\nSpaces

In [48]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [49]:
create_brochure("HuggingFace", "https://huggingface.com")

/opt/anaconda3/envs/llms/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/anaconda3/envs/llms/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/anaconda3/envs/llms/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/anaconda3/envs/llms/l

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.com/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.com/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.com/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.com/blog'}, {'type': 'contact page', 'url': 'https://huggingface.com/join'}, {'type': 'discussion forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'social media', 'url': 'https://twitter.com/huggingface'}, {'type': 'social media', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


/opt/anaconda3/envs/llms/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/anaconda3/envs/llms/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/anaconda3/envs/llms/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apply.workable.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/anaconda3/envs/llm

```markdown
# Welcome to Hugging Face! 🤗

## The AI Community Building the Future (No Robots Allowed...yet)

### Who Are We? 
We’re the coolest kids in the AI playground! At Hugging Face, we’re building a thriving community of machine learning enthusiasts who collaborate on models, datasets, and applications. Think of us as the friendly neighborhood Spider-Man, but for AI—saving the world one algorithm at a time!

### What Do We Do? 
- **Models**: We host a whopping **1M+ models** including trending stars like **meta-llama/Llama-4-Scout**! (Can you tell we love llamas?)
- **Datasets**: Browse through **250k+ datasets** of information so rich, they make Scrooge McDuck look poor. 
- **Spaces**: Join **3.59k running applications** that will make you go "Whoa!" (or at least "Ooh!") 
- **Enterprise Solutions**: Got a team? We offer the most advanced platform with minor details like superb security and dedicated support. 

### Our Culture
At Hugging Face, we embrace collaboration like a warm hug after a long day. We value teamwork, creativity, and the occasional meme-sharing session over coffee. If you love the idea of working with passionate individuals who are as obsessed with AI as they are with cat videos, you’ll feel right at home.

### Who are Our Customers?
Our enterprise solutions have won the hearts of **50,000+ organizations**, including:
- Tech Giants (Hello, Google, Microsoft, and Amazon!)
- Non-Profits (because we like to save the world, not just code it)

### Careers at Hugging Face
We're always on the lookout for talented individuals! If making friends with algorithms and growing your machine learning portfolio sounds enticing, **join us!** You'll work on cutting-edge technology that can even turn your dreams into code (well, not literally... we haven’t cracked that yet).  
🔍 **Check our Current Openings!**

### In Conclusion...
At **Hugging Face**, we don’t just build AI—we build a community! So, whether you want to deepen your knowledge, share your work, or jump on board our spaceship to AI galore, we can’t wait for you to join our adventure! 

Hugging Face: Where the future isn’t just bright—it’s downright brilliant! 🌟

> **P.S.** We promise, not all our employees are AI models. Many of us are real people. (Can confirm, I just drank coffee…lots of it.)
```


## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [50]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [51]:
stream_brochure("HuggingFace", "https://huggingface.co")

/opt/anaconda3/envs/llms/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/anaconda3/envs/llms/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


/opt/anaconda3/envs/llms/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/anaconda3/envs/llms/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apply.workable.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/anaconda3/envs/llms/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/anaconda3/envs/llms

# 🤗 Hugging Face Brochure: Embrace the Future with a Smile! 🌟

## Welcome to Hugging Face: The AI Playground!

Where else can you find 1 million+ models ready for a good hug? 🤗 Here at Hugging Face, we're not just another tech company; we’re a thriving AI community building the future one (hug) model at a time! Think of us as the warm, fuzzy embrace of the machine learning world!

---

### 🤖 What We Do: 
- **Models Galore!**: From meta-llama 🦙 to DeepSeek 🐳, we have models trending faster than your favorite celebrity! And yes, they are better than many reality TV shows. 
- **Datasets for Days**: 250,000+ datasets just waiting to be explored — because who doesn’t love a good data binge?
- **Spaces that Spark Joy**: Create your own AI applications or simply lurk in the vast ocean of creativity with 400,000+ applications running around like it's a digital summer party!

---

### 🏢 Company Culture: The Hugging Community
- **Open Source Love**: We believe in sharing! Over 50,000 organizations from AI2, Amazon, and Microsoft 🏢 are riding the Hugging Face wave (and promoting that teamwork makes the dream work).
- **Unlimited Hugs**: Collaboration isn’t just a buzzword here; it’s a way of life! Whether you’re sharing a model, dataset, or an occasional meme, we make it all about community and joy!
- **Open Doors**: Are you passionate about machine learning? Great! You can host unlimited public models, datasets, and applications. 

---

### 🌈 Why Choose Hugging Face? 
- **Boost Your ML Skills**: We don’t just help you LEARN machine learning; we help you thrive, like a well-watered plant (but without the brown thumb).
- **Paid Compute Solutions**: Starting at $0.60/hour — yes, folks, you can deploy your epic models without breaking the bank! And enterprise solutions starting at just $20/user/month: because embracing innovation shouldn't cost an arm and a leg.

---

### 🚀 Career Opportunities: Join the Hugging Face Family!
Join us if:
- You adore machines treating you like one of their own! 🤖
- You’re ready to open-source your heart and soul—pip install new friendships included.
- You want to be a part of something big, fun, and impactful in the AI community. 

### 🌍 Hop on the Hugging Train!
Don’t be a stranger! Bring your creativity, curiosity, and quirkiness to Hugging Face. Because here, we believe exploring AI should be as fun as building a pillow fort! 🏰

---

**Connect with Us:**
- **Website**: [Hugging Face](https://huggingface.co)
- **Get Social**: Follow us on Twitter, GitHub, and LinkedIn!
- **Chat with Us**: We’ve got a Discord for all your burning questions (no, it’s not just for memes)!

Join us and let's shape the future of AI together—with all the hugs! 🤗

In [52]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

/opt/anaconda3/envs/llms/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/anaconda3/envs/llms/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'discussion forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


/opt/anaconda3/envs/llms/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/anaconda3/envs/llms/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apply.workable.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/anaconda3/envs/llms/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/anaconda3/envs/llms

# Welcome to Hugging Face!

**Where AI Dreams Come True! 😄🤖**

---

### What’s Hugging Face?

At Hugging Face, we believe in a world where humans and machines can hug it out! 🫂 As a vibrant AI community, we empower developers to create, discover, and collaborate on cutting-edge machine learning models, datasets, and applications. Together, we’re building the future—one model at a time!

---

### What’s Cooking? 🔥

**Trending Models** 
- **Llama-4-Scout-17B-16E-Instruct**: Updated 9 hours ago with 133k thumbs ups! (Don’t worry, the llama won't bite!)
- **DeepSeek-V3-0324**: Trending with a double-shot of shout-outs - 175k & counting! 

**Space Explorers** 
- Want to dive into AI apps? From **Ghibli EasyControl** to **Hi3DGen**, our spaces are bustling like a bee at a honey convention! 🍯🐝

---

### Our Customers: The Who & The Wow! 🥳

Hugging Face is the go-to platform for over **50,000 organizations** (and counting)! From Google to Amazon, they can't resist our charm! Our customers are basically the cool kids' club of the AI community. 🏆✨

---

### Careers: Join the Hug Squad! 🚀

Looking for a place where you can flex your AI muscles? At Hugging Face, we cultivate a culture that’s as open and welcoming as a giant bear hug! 🐻❤️ Here’s what to expect:
- **Collaborative Vibes**: Join forces with top-notch engineers and data wizards.
- **Innovative Spirit**: Work on state-of-the-art ML tools (ever heard of Transformers? We’ve got ’em 142,679 times!).
- **Flexible Environments**: Choose your work style, whether it’s from home, in the office, or on the moon!

---

### Join Us on the Journey! 🌍🚀

So, whether you’re a curious data scientist, an AI guru, or just someone who really loves hugging (no judgment here), come join the Hugging Face family! Together, we’ll reshape the world of AI one cuddle at a time!

- **Explore**: [AI Apps](https://huggingface.co/explore)
- **Datasets**: Need data? We’ve got over **250k options**! 
- **Models**: With **1M+ models**, you’ll never have a boring day again!

---

### Ready to Hug It Out? 

With Hugging Face, the future is bright and the models are even brighter. Dive in, collaborate, and watch your ideas transform into reality! 🌈✨

**Contact Us**: [Hugging Face](https://huggingface.co/contact)

--- 

*Disclaimer: No actual hugging is required, though high-fives are encouraged! 🖐️*

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

In [53]:
stream_brochure("Motorola", "https://motorola.com")

/opt/anaconda3/envs/llms/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'motorola.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/anaconda3/envs/llms/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.motorola.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/anaconda3/envs/llms/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.motorola.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/anaconda3/envs/llms/l

Found links: {'links': [{'type': 'about page', 'url': 'https://www.motorola.com/us/en/about'}, {'type': 'executive team page', 'url': 'https://www.motorola.com/us/en/about/motorola-executive-team.html'}, {'type': 'business page', 'url': 'https://www.motorola.com/us/en/business.html'}, {'type': 'careers page', 'url': 'https://www.motorola.com/us/en/careers'}]}


/opt/anaconda3/envs/llms/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.motorola.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/anaconda3/envs/llms/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.motorola.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/anaconda3/envs/llms/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.motorola.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/anaconda3/envs/ll


# 🎉 Welcome to Motorola: Where Smartphones Meet Sassy Innovation 🎉

## Who We Are
Founded in 1928 (and somehow still cool!), Motorola is passionate about **designing a world where you can connect, share, and creatively express yourself**. Our motto? If it doesn't have a signal, what's the point? Our phones range from the iconic **Razr** to the powerful **Edge**, so you can communicate your way – whether it’s through late-night TikTok videos or ultra-important group chats!

## Company Culture: More Than Just Pixels and Pixels 🎨
At Motorola, we believe in fostering innovation that inspires – like giving away **FREE Bose earbuds** with select purchases! Our team enjoys a culture of **collaboration, creativity, and the occasional legendary office ping pong battle**. Forget the suits; we want you rocking those stylish Moto tees while you brainstorm your next big idea.

## Customers: From Tech-Wizards to Business Tycoons 👩‍💻👨‍💼
Our products cater to everyone! Whether you’re a **tech novice** looking for a simple phone to call grandma or a **business mogul** needing our security-savvy *Thinkshield* for sensitive corporate communications, Motorola has your back. We even help you give your nursery that high-tech security you never knew you needed! 

## Humble Brag Corner: Our **Products** 🛍️
From smartphones that flip open like a ninja to accessories that even your cat would approve, here’s what we offer:
- **Razr Family**: Get nostalgic while staying modern. Flip your way to style, folks!
- **Motorola Edge Family**: Cutting-edge tech without the paper cuts!
- **Moto G Family**: Affordable yet powerful. This is where value and style meet like long-lost friends.
- **Smart Home Devices**: Who says your home can’t be smarter than you? We help unlock the new age of smart living – complete with baby monitors to keep an eye on your precious mini-humans!

## Careers: Join the Fun! 🌟
Looking for a career that doesn’t suck out your soul like a black hole? Look no further!
- **Why Work Here?**: What more could you want? Tech, creativity, and the joy of making people smile when they flip open their phones.
- **Open Positions**: We’re on the lookout for the wildly innovative, the obsessively collaborative, and those who openly communicate with their gadgets. If you can charm a smartphone into working properly, you might fit right in!

## Sales & Discounts: We Love Deals! 🤑
- Don't miss our **Tax Season Sale**: Discounts so juicy even your accountant might crack a smile. 
  - **Razr+ 2024**: $200 off (We're serious! Look it up!)
  - **Moto Edge+**: $400 OFF – better than a shopping spree!
  - **Moto G Stylus**: Get a *FREE Bose Earbud* with your purchase. Who knew multitasking could sound so good?

## Join Us in Innovating the Future 🌈
So, if you're seeking a **smartphone that’s smarter than your ex**, or considering a career with a company that values creativity and laughter, **Motorola is the place for you**! 

Have questions or want to explore? Just flip us a message, or visit our website for more information!

🚀 **Motorola: Where Innovation Meets Fun. Let's Connect!**



In [54]:
stream_brochure("Motorola", "https://motorola.com")

/opt/anaconda3/envs/llms/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'motorola.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/anaconda3/envs/llms/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.motorola.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/anaconda3/envs/llms/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.motorola.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/anaconda3/envs/llms/l

Found links: {'links': [{'type': 'about page', 'url': 'https://www.motorola.com/us/en/about'}, {'type': 'about page - executive team', 'url': 'https://www.motorola.com/us/en/about/motorola-executive-team.html'}, {'type': 'business page', 'url': 'https://www.motorola.com/us/en/business.html'}, {'type': 'careers page', 'url': 'https://www.motorola.com/us/en/financing.html'}]}


/opt/anaconda3/envs/llms/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.motorola.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/anaconda3/envs/llms/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.motorola.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/anaconda3/envs/llms/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.motorola.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/anaconda3/envs/ll

# 📱 Welcome to Motorola: Where Cool Meets Connectivity! 📱

---

## About Us: 
At Motorola, we’ve been wired into the world since 1928! We design innovative products that help you connect, share, and enjoy life to the fullest. In other words, we’ve been ‘mobility-ing’ before it was cool. Our mission is simple: to make your everyday life easier, more uplifting, and most definitely more fun. 

---

## Our Product Lines: Because Regular Phones Are Just So Last Century!

### Razr Family
**Flip it like it’s hot!** The Razr redefines the cool-factor with its foldable design and slick features. Seriously, you’ll make *everyone* turn their heads.

### Edge Family
**Edge on the fun!** Whether you want crystal-clear selfies or want to show off your gaming skills, our Edge line does it all—just beware, don’t take it for granted, or it just might edge *away*!

### Moto G Family
**Get your G on!** Seriously, they’re so good, we gave them a letter. And all over the world, they are here to help you express yourself—affordably!

### Smart Home Devices & Accessories
From security systems that’ll make burglars rethink their career choices, to headphones that'll drown out your neighbor’s karaoke nights, we got your back!

---

## 📊 Deals so Good, They’re Practically a Joke! 
- **razr+ 2024** - Get ready to flip your wallet because it’s $200 OFF! 
- **Moto edge+** - Hold onto your pants, it’s a whopping **$400 OFF**! 
- **moto g stylus** - As if a smartphone wasn’t enough, it comes with FREE Bose earbuds! (Yes, please!)

## **TAX SEASON SALE** 
Because who said tax season can’t be fun?

---

## 📣 Moto for Business & Thinkshield
**Caution!** High-performance productivity ahead! We cater to businesses of all sizes. Our Thinkshield revolutionizes how companies secure their data—no superhero cape required, but you might feel like one!

---

## 🌈 Culture Corner: Join the Motorola Family!
We love innovation, we love people, we love... puns. Our team is all about creativity, collaboration, and the occasional ping pong showdown. When you work here, you’re not just an employee; you’re a part of a groundbreaking legacy! 

### Careers
**Looking For a Job?** 
Join us if:
- You think "Thinkshield" is the next best superhero in the lineup.
- You can make phone jokes that even your mom will laugh at. 
- You’d like to work in a place that encourages flip phone nostalgia!

---

## 🌍 Our Customers: Individuals Who Like to Have Fun
From young professionals snapping selfies on the go, to parents keeping an eye on their little ones with our smart nursery monitors, we cater to those who understand that life is a wonderful mixture of connections, fun, and occasional phone drops!

---

## 😄 Closing Joke:
Why did the smartphone go to school?
Because it wanted to improve its “cell-f” esteem! 

---

**Motorola:** Connecting the world, one smartphone at a time! So come on down and find your next conversation starter today!

In [56]:
stream_brochure("OneMain Financial", "https://www.onemainfinancial.com/")

/opt/anaconda3/envs/llms/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.onemainfinancial.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/anaconda3/envs/llms/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.onemainfinancial.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Found links: {'links': [{'type': 'about page', 'url': 'https://www.onemainfinancial.com/about-us'}, {'type': 'careers page', 'url': 'https://jobs.onemainfinancial.com'}, {'type': 'contact page', 'url': 'https://www.onemainfinancial.com/contact-us'}, {'type': 'resources page', 'url': 'https://www.onemainfinancial.com/resources'}, {'type': 'personal loans page', 'url': 'https://www.onemainfinancial.com/personal-loans'}, {'type': 'credit cards page', 'url': 'https://www.onemainfinancial.com/credit-cards'}, {'type': 'mortgage payment solutions page', 'url': 'https://www.onemainfinancial.com/mortgage-payment-solutions'}, {'type': 'reviews page', 'url': 'https://www.onemainfinancial.com/reviews'}, {'type': 'military benefits page', 'url': 'https://www.onemainfinancial.com/military-benefits'}, {'type': 'faq page', 'url': 'https://www.onemainfinancial.com/faq'}]}


/opt/anaconda3/envs/llms/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.onemainfinancial.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/anaconda3/envs/llms/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'jobs.onemainfinancial.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/anaconda3/envs/llms/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.onemainfinancial.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.war


# Welcome to OneMain Financial: Where Personal Loan Dreams Come True!

## 💰 Discover Money That Moves You
At OneMain Financial, we believe that **every penny counts**! Need a little cash? Whether it's to pay off those pesky bills or to fund your next big adventure (skydiving lessons, anyone?), we've got your back with **personal and auto loans from $1,500 to $20,000**! 

### Featured Perks:
- **Quick Decisions**: Get money in hand as fast as 1 hour after loan closing. (Yes, that’s faster than you can find matching socks!)
- **Real Humans, Not Robots**: Got questions? Our friendly team of branch employees, phone gurus, and online wizards are here to help! 

## 🎊 We're in Your Corner!
With tens of thousands of happy customers, we're officially the **lender you can lean on**. But don't just take our word for it; our loan products are as easy as pie (and we don’t mean the fruit kind)!

### What Makes Us Special:
- **Fixed Payments**: Clear, easy-to-understand terms that won't give you a headache. No surprises here—except maybe good ones!
- **Budget Mastery**: Access resources that help you stretch your budget further than you thought possible (like in that movie where someone stretches a car!). 
- **BrightWay Credit Cards**: Get rewarded for simply being you and paying your bills on time. Turning payments into progress? Now that’s our jam!

## 🌟 Company Culture: Live the OneMain Life!
Think corporate means boring? Think again! Our culture is a mix of enthusiasm, support, and lots of spontaneous potlucks. We believe in our mission and in **serving our community**. When we say we’re here for you, we mean it (and yes, we will also listen to your playlist while we help)!

### Career Opportunities
Looking to join a team that values you? We've got room for **creative minds, problem solvers, and financing prodigies**. Plus, there are plenty of snacks (and sometimes celebratory dance-offs)! Check our [Careers page](https://www.onemainfinancial.com/careers) to apply and unleash your potential with us!

## 📞 Let’s Connect!
Spotted something that tickles your financial fancy? Need assistance? Reach out at **800-290-7002** or visit one of our many branches. Our supportive team can't wait to hear from you!

---

### **Disclaimer**: 
Funds availability may depend on a variety of factors, and we cannot guarantee loan amounts for those desiring funds for less-than-savory activities (sorry, no gambling or buying mysterious cryptocurrency). 

---

**Ready to take control of your financial fate?** Apply today at [OneMain Financial](https://www.onemainfinancial.com) because every adventure needs a reliable co-pilot!


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>